# PatMatch Use on Custom Sequences and Integrating with Python 

The [previous notebook](PatMatch initial demo and introduction.ipynb) used prepared data supplied by the software authors in a `test` directory. The web-based PatMatch offerings listed [here](https://github.com/fomightez/patmatch-binder/#usage) lock you into matching patterns to specific sequencess. With the stand-alone, command line based PatMatch, you can run pattern matching on any sequence you'd like. Here, we will start at square one and work through such an example. 

This will get example sequence data from an external source, prepare it, and then analyze it. Then, subsequent steps will parse the resulting data into something useful in Python. And, even cover how to convert to Excel. 

### Preparing to use PatMatch software on raw sequence data

First, example nucleic acid sequence to work with is retrieved. The sequence files PatMatch works with are FASTA according to the [USAGE](PatMatch%20initial%20demo%20and%20introduction.ipynb#Usage) information.

Click on the cell below and type `shift-enter` or press `Run` on the toolbar above to get an example file.  
(In Jupyter notebooks running in the Python kernel as indicated in the upper right of this notebook, commands to the shell are prefaced with exclamation points. You'll notice when we switch to dealing with Python directly, this will not be needed.)

In [ ]:
!curl -O https://downloads.yeastgenome.org/sequence/S288C_reference/chromosomes/fasta/chrmt.fsa

<font color="#999">(Alternative ways to import data using the Jupyter environment's graphical user interface will be covered below.)</font>

Check the file listing by executing the next cell to see the FASTA-formatted file has been retrieved. Work through the following cells simialrly.

In [ ]:
!ls

Further following the PatMatch [USAGE](PatMatch%20initial%20demo%20and%20introduction.ipynb#Usage) information, sequences should be processed so that the lines of sequence data are formatted to one line for handling by PatMatch. The PatMatch authors have provided a utility script for doing that preparation step. The following cell will run that on the example data.

In [ ]:
!perl ../patmatch_1.2/unjustify_fasta.pl chrmt.fsa

That will produce a file with `.prepared` appended to the end of the supplied file name. 

Check that file was produced by running checking the file listing again using `!ls`.

In [ ]:
!ls

Having verified the prepared data file exists, you are ready to run the program to search for a pattern.

### Running PatMatch

The PatMatch [USAGE](PatMatch%20initial%20demo%20and%20introduction.ipynb#Usage) information says `-n` is for nucleotide pattern match and `-c` is for complementary strand; however, [based on my tests](PatMatch%20nucleic%20handling%20flags%20demystified.ipynb) it seems that `-c` means it is for the complementary strand **in addition to** the strand in the dataset. 

**<font color="red">Therefore, if you want the pattern search to be performed on BOTH strands of the supplied sequence, as is the default of the web-based PatMatch tools, you actually want to use the `-c` flag when authoring the command.</font>**

If you are curious about this aspect futher, I demonstrate that [here](PatMatch nucleic handling flags demystified.ipynb) and in the course of that cover how to replicate the three options typically offered for strand at PatMatch web-based offerings. Feel free to examine and run that notebook or simply use the `-c` flag if you are trying to scan both strands. 


In [ ]:
# !perl ../patmatch_1.2/patmatch.pl -n "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared #dataset strand only
!perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared

In [ ]:
!ls

That is the basics of running PatMatch. There are options you can add to control this mismatch amount and whether to allow insertions,deletiions, or substitutions towards thos mismtaches. Example:

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared 1 ids

See the [USAGE](PatMatch%20initial%20demo%20and%20introduction.ipynb#Usage) for more information about those options. However, that covers the basics.

With the basics in hand, and using the power of the command line, searches of more sequences or more sequences and more patterns become possible. However, you'll quickly encounter problems handling all those results. As a simple example, we'll use the example pattern matching search we developed above as example for integrating with Python for more efficient handling of the results and to touch upon the advantanges offered by combining with a scripting language.

--------


## Importing PatMatch Results into a Pandas Dataframe and Exporting to Excel

Now that you see what PatMatch is returning as results, you'll probably note that while that looks easy to read for a human, it isn't very computer friendly. Indeed, if you have used the web-based PatMatch offerings, you'll note that they return the results in a table form that is more useful. I've made a utility script to do that. It has a lot of bells-and-whistles to make it convenient to use, but its it core it just picks apart the results from PatMatch to make a table. It makes an improved table if you provide it with some optional information it won't have access to otherwise, namely the pattern you searched for and a name for that pattern. 

Let's make some data to parse using the sequence we imported and prepared above and see what the script returns. 

Note that above we just ran the script and saw the results as output. Now you'll be saving the results to a file this time to pass the data to the python script. (Advanced notebooks in this series will show other ways to do this passing, but saving results to a file for use in the next step in a workflow is a common idiom for computational biology.)

Run the next cell and continue running the subsequent steps to go through the process.

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared > test.out

Once that finishes running, it will get a number in front of it and we will have a file to process. You won't see output below the call because we directed the output from the PatMatch command to a file by following the command with `>` followed by the file name, `test.out`, where we wanted output written. (This is taking advantage of what is called 'redirection' in the shell.)

But we need the script to process it with. Run the next cell to obtain that script.

In [ ]:
!curl -O https://raw.githubusercontent.com/fomightez/sequencework/master/patmatch-utilities/patmatch_results_to_df.py

That command will have saved the script to the machine where this notebook is running. The reason you are obtaining it from the source, and it wasn't just included in this repository is to insure you have a working version. At the time of writing this, that script was nascent and presumably would be adjusted further. If I was interested here in best practices for reproducibility, and I knew that script was fairly established, I would have been better off doing one of two things:

* include in the repository the version of the code specifically used to get the results shown in this notebook
* obtain the version of the code that was used at that time by referencing the time-stamped version in the curl command (or `%load` command).

However, this notebook is simply to demonstrate steps and options possible and not document 'results' obtained, and so we are just obtaining the most recent version.

Finally with a results and the script, you are ready to move the simple text results to a Python object. Let's call the script to act on the results in a simple way to see what happens and then build on that after.

In [ ]:
%run patmatch_results_to_df.py test.out

Success!

(If you don't see it working, chances are you skipped one of the earlier steps. Just run the commands above again. The easiest way to do that is select `Run All Above` from under the `Cell` menu on the toolbar.)

However, we can see right away it could use improvement. For example, the question marks under `query pattrn` would look better with the pattern you had used in the search. So run the script with the `--help` flag  (simply `-h`, alternatively) to see if it will tell you anything more about what the script expects and how we might better pass information in your command.

In [ ]:
%run patmatch_results_to_df.py --help

That will return a detailed set of documentation about the script and how to use it. Alternatively, looking at the README from [where you got the script](https://github.com/fomightez/sequencework/tree/master/patmatch-utilities) would have helped you as well. In fact the README I wrote is probably easier to understand if you are new to Python scripts because the way the options are shown in the `usage` follows a convention that may not be familiar yet. You'll see probably what you want when calling the script is more like what is on the next line. Run that.

In [ ]:
%run patmatch_results_to_df.py test.out --pattern DDWDWTAWAAGTARTADDDD -name promoter

Now to convince you this is useful...

Because I suggested you do this a way that is more common to more computational systems, I have made my work harder. If I had relied on the Jupyter environment, we'd already have that data as an active Python object in memory. However, right now it is stored in a file that isn't even human readable. If you downloaded that file it says it stored and tried to open it in a text editor, you'd see a lot of weird characters like if you tried to open a Microsoft Word document in a simple text editor and not Word.

Why am I telling you this? A: First, I am hoping that you'll read on in this series of notebooks and see how when using the Jupyter environment there is an easier way to use this script to get the dataframe as a Python object in memory without the intermediate step we'll be performing here. Second, I am trying to barrel through what may be some intermediate-level Python because of a design choice I made because of how at the time I saw this script fitting in my analyses. 

Why didn't I just have it make a save a text or Excel table as output from the script? A: To be clear, we will make both of those soon. I could have easily had the script save it as text or tab-delimited file, but that wouldn't be much more useful than what you can get by copying a pasting the table-like structure that was formed as it was running. Moreover, that path takes you away from Python to where you have have less power. To use Python with it you'd still have to read it back in and the way I saved it made reading it back in immensely easier. Similarly, to have saved it as an Excel it would have added another dependency to my script which limits where it can easily be run. Often it is best to keep things simple. Additionally, for my immediate use I needed a dataframe and you'll see it is easy enough to make any of thee other stuff with that in hand.

Well rely on Python's Pandas package to easily work with the dataframe we have and do all those things and more. Pandas is a module that is not in the core of Python but adds a lot of useful abilities. Pandas is already installed here in this Jupyter environment if you have gotten this far.

Next't you'll use that module to read out stored dataframe back in to active memory by running the next cell. Just before running the command to do that we import the Pandas module into memory to make all the associated methods and functions available in memory.

### Back to the Python

In [ ]:
import pandas as pd
df = pd.read_pickle("patmatch_pickled_df.pkl")

That cell should complete in an instant and now the dataframe is in active memory.

Note that that command `df = pd.read_pickle("patmatch_pickled_df.pkl")` will do the same thing if built into a Python script or run in a Python/IPython interpreter window. (Pandas needs to be installed and imported in those places.) It isn't particular to the Jupter environment in any way. And, that is indeeed why for the first example of chaining PatMatch to Python, I relied on the intermediate file-saving step. We'll see it is easier if we are in Jupyter later. On the other hand, the steps we have done so far we'll work anywhere since Python is almost universally installed on systems with command lines. You may just need to install Pandas with a simple `pip install pandas` entered directly on your command line. 

Now the data is no longer just in the file. It is available as Python objects/data in this notebook.
What does that get you?

* data that can be easily used in Python 

* data that can be easily translated to text tables or even Excel

* nicely styled dataframe we can use in Jupyter

Let's demonstrate **all** those things. 


Okay, I said now the dataframe is active memory.
You can demonstrate the dataframe is in active memory by viewing it and seeing a nicely styled dataframe (when in a Jupyter notebooks):

In [ ]:
df

As another proof you can run some of the methods associated with a Pandas dataframe.

In [ ]:
df.describe()

In this particular case, beyond the count, those statistics available don't really mean anything but you can imagine if you hae a really dataframe of quantified data you'd have just displayed a useful summary.

In [ ]:
df.tail()

### Demonstrating the Utility of Having the PatMatch Data in Python


I mentioned above that now the data is in a useful form that can be used in Python.

To demonstrate that, you can further explore what the dataframe object and Pandas can do by extracting some of the data. In this example, imagine that the potentially matching pattern `ATTAATATAAGTAATATATA` was particulaly important and we wanted to identify if it was there.

In [ ]:
special = df[df['matching pattern'] == "ATTAATATAAGTAATATATA"]
special

Or imagine we were particularly concerned with any cases on the reverse strand:

In [ ]:
reverse = df[df['strand'] < 1]
reverse

What if we wanted to send to a list all the coordinates where the patterns on the reverse strand begin and use a simple Python `print` command to see the contents of that list:

In [ ]:
rev_starts = reverse['start'].tolist()
print (rev_starts)

Utilizing the dataframe with other scripts is now possible, too. Go ahead and retrieve a script from Github, bring the main function into the active notebook environment, and then prepare to use the dataframe with that function.

In [ ]:
!curl -O https://raw.githubusercontent.com/fomightez/sequencework/master/plot_sites/plot_sites_position_across_chromosome.py
from plot_sites_position_across_chromosome import plot_sites_position_across_chromosome 

After running that command above, you can now call the main function of the script, `plot_sites_position_across_chromosome()`, and give it the dataframe.

The code to do that would be:
```
%matplotlib notebook
sites_df = df
plot_sites_position_across_chromosome(sites_df)
```

However, if you were to go ahead and try that now, you'll see `KeyError: 'sys_gene_id'` because the column headings in the current dataframe don't presently match up with what the function expects.

So first we need to fix the dataframe to match what the function is expecting. We are going to convert the `hit_id` column to be titled `sys_gene_id` to make things sync with what the function is seeking.

In [ ]:
df = df.rename(columns={'hit_id':'sys_gene_id'})

In [ ]:
%matplotlib notebook
sites_df = df
plot_sites_position_across_chromosome(sites_df);

You should see a plot. (If by any chance you don't see a plot and also don't see an error, just try running the cell again.) But it isn't much to look at because there is only one type of sequence element. 

However, by combining steps seen above and a little more power of Pandas to concatenate the dataframes, you can quickly add a lot of other sites easily.

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -c "GAATTC" chrmt.fsa.prepared > ecori.out
!perl ../patmatch_1.2/patmatch.pl -c "GGATCC" chrmt.fsa.prepared > bamhi.out
!perl ../patmatch_1.2/patmatch.pl -c "TCTAGA" chrmt.fsa.prepared > xbai.out
%run patmatch_results_to_df.py ecori.out --pattern GAATTC -name EcoRI -dfo ecori_df.pkl
%run patmatch_results_to_df.py bamhi.out --pattern GGATCC -name BamHI -dfo bamhi_df.pkl
%run patmatch_results_to_df.py xbai.out --pattern TCTAGA -name XbaI -dfo xbai_df.pkl
ecori_df = pd.read_pickle("ecori_df.pkl")
bamhi_df = pd.read_pickle("bamhi_df.pkl")
xbai_df = pd.read_pickle("xbai_df.pkl")
df_res = pd.concat([ecori_df,bamhi_df,xbai_df], ignore_index=True)
df_res = df_res.rename(columns={'hit_id':'sys_gene_id'})
df_new = pd.concat([df,df_res], ignore_index=True)
%matplotlib notebook
sites_df = df_new
plot_sites_position_across_chromosome(sites_df);

We won't go further here but if you chain more of such steps together to make a useful script or set of commands in a Jupyter notebook that would allow you to easily analyze results in a dataframe using Python. Dataframes really provide a feature-rich way to store and work with data in Python. And that is why I made a script to move PatMatch results to such a form.

Let's finish by moving the PatMatch results data from the dataframe to a tab-delimited text table and to Excel.

### Saving in table form

The Pandas module makes it really easy to save in a delimited style, such as tab-delimited or comma-delited. You can save the data as a tab-delimited file, a.k.a. tab-seperated values file, with the next command.

In [ ]:
df.to_csv('example.tsv', sep='\t',index = False) 

That file can easily be downloaded via the route discussed in the next section and then can be easily opened with files that handle spreads sheeets, like Excel. (In fact, this is the more 'universal' way to store and share data, but because that command can easily be performed later when results are finalized, I left it out of the `patmatch_results_to_df.py` for now.) You could easily later go back from that tab-separated file to a dataframe with the command `df = pd.read_csv('example.tsv', sep='\t')`.

If you work with people that use Excel regularly, you can even go right to an Excel file from the dataframe as well. You'll need a less commom Python package to do this. The next command will install that in this Jupyter environment.

In [ ]:
!pip install openpyxl

With `openpyxl` installed, you can use a similar command as when you made the tab-delimited file to save the dataframe as an Excel table.

In [ ]:
df.to_excel('example.xlsx')

You have to download the Excel file to view it, see the next section.

------

Grab anything useful, or add items you want to scan
------------------------------------------------

As written, this notebook just uses test sequences, yet hopefully you can easily see how to adapt things to work with your own data and may have already started generating useful information. Because this is being run in an ephemeral remote instance, you'll want to download any realy data you generate ASAP. **Bear in mind, if you are inactive for more than 8 minutes the instance can be culled. Treat this computing resource as temporary and highly unstable.**

You can go to the Jupyter dashboard by pressing the Jupyter logo in the upper left side of this notebook. There you'll find a handy GUI interface to allow downloading of files. Because this particular notebook was run in the `notebooks` directory, you have to select that folder and navigate into it first before you'll see the files you made with the steps above. Some of the text files made can even be viewed in that file viewing directory. However, you'll have to download the Excel file to see if it worked as there is no rendering/viewing built in. This interface can also be used to upload your own data if it is not available on the web or you simply prefer a GUI interface over using shell or Python commands to retrieve data.

In addition to downloading any generated tables, you can download any generated pickled dataframes. They can be uploaded to a newly launched Jupyter environment later and accessed using the commands illustrated above.

In fact as a further plug for the style of storage of the dataframe used here:

Pickled dataframes 

* pickling a dataframe can provide snapshot of the data as it was at that time.
* saves crafting a command to read in from text because stores everything as it was. 
* Python 2.7 can directly read those made in Python 3 but you can convert to text and then back again in the appropriate environemnt.

Pickling is a nice alternative to make a snapshot that is useful without need to go back to square one. Because of the way we did things the way we did in this notebook, it would easy enough to run from scratch to bring it back into memory. You can imagine though as you do real analysis things will change over time and you may like to go back to an older result to see what changed or use a simpler form for developing downstream steps. Or, of course, it can just be a **convenient way to not have to go back to square one when you want to return to an analysis after working on something else**.




Next
----

Return to the [index](../index.ipynb) or continue on with the next notebook to see demonstration of working with command line-based PatMatch with a genome and protein sequence data. The notebook following that one will demonstrate a way to skip having the script save and then immediately bring that file back in to Python when you are actively perfoming pattern matching.